In [104]:
opts = {
    'published_at_start': 'NOW-2DAYS',
    'published_at_end': 'NOW',
    'language': ['en'],
    'per_page': 5,
    'source_locations_country': ['US'],
    'categories_taxonomy': 'aylien',
    'aql': 'categories: {{taxonomy: aylien AND id:ay.sports.nba}}'
}

In [2]:
import os
from dotenv import load_dotenv
from __future__ import print_function
import time
import aylien_news_api
from aylien_news_api.rest import ApiException
from pprint import pprint

load_dotenv()
API_ID = os.getenv("AYLIEN_API_ID")
API_KEY = os.getenv("AYLIEN_API_KEY")

configuration = aylien_news_api.Configuration()
configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = API_ID
configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = API_KEY

client = aylien_news_api.ApiClient(configuration)
api_instance = aylien_news_api.DefaultApi(client)

try:
    api_response = api_instance.list_stories(**opts)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'next_page_cursor': 'MTY3NzE3MjQ0MDAwMCw1MzMzMjIxOTM1',
 'published_at_end': datetime.datetime(2023, 2, 23, 17, 20, 14, 666285, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 2, 21, 17, 20, 14, 659567, tzinfo=tzutc()),
 'stories': [{'author': {'avatar_url': None,
                         'id': 29395147,
                         'name': 'Matt Peralta'},
              'body': "D'Angelo Russell's first stint with the Los Angeles "
                      "Lakers didn't go according to plan as the organization "
                      'soured on him quickly.\n'
                      ' Although Russell was clearly a talented player, the '
                      'organization had issues with his inconsistent play and '
                      'his bouts of immaturity. With a new front office regime '
                      'led by Magic Johnson, the Lakers selected Lonzo Ball '
                      'No. 2 overall in the 2017 NBA Draft. Johnson cited a '
                      'nee

In [318]:
s1 = api_response.stories[0]

def convert_story(story):
    converted = {
        'text': get_summary(story.body, 8), 
        'entrities': format_entities(story.entities), 
        'links': story.links.permalink,
        'authors': story.author.name,
        'thumbnails':  story.media[0].url if len(story.media) > 0 else None
    }
#     print(converted)
#     converted['text'], converted['entrities'], converted['links'] = , , story.links.permalink
    return converted

# def get_summary(story):
#     return story.body if len(story.body) < 2500 else ", ".join(story.summary.sentences)

def format_entities(ents):
    ents_output = [
        (entity.body.surface_forms[0].text, entity.types[0], entity.overall_frequency)
        for entity in ents
        if len(entity.body.surface_forms) > 0 and len(entity.types) > 0 and isinstance(entity.overall_frequency, (int, float))
    ]
    
    if len(ents_output) > 30:
        ents_output = [item[:2] for item in ents_output if item[2] > 1]
    else:
        ents_output = [item[:2] for item in ents_output]
        
    return ents_output

# ents = format_entities(s1.entities)

# len(s1_summary), s1_summary, ents

In [281]:

# full_article

In [114]:
opts['story_id'], opts['per_page'], opts['not_id'] = s1.id, 2, [s1.id]

# per_page for related doesn't work always returns 10

try:
    related_response = api_instance.list_related_stories_get(**opts)
    pprint(related_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'published_at_end': datetime.datetime(2023, 2, 23, 19, 15, 59, 154968, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 2, 21, 19, 15, 59, 154900, tzinfo=tzutc()),
 'related_stories': [{'author': {'avatar_url': None,
                                 'id': 3583220,
                                 'name': 'Matt Peralta'},
                      'body': "D'Angelo Russell's first stint with the Los "
                              "Angeles Lakers didn't go according to plan as "
                              'the organization soured on him quickly.\n'
                              ' Although Russell was clearly a talented '
                              'player, the organization had issues with his '
                              'inconsistent play and his bouts of immaturity. '
                              'With a new front office regime led by Magic '
                              'Johnson, the Lakers selected Lonzo Ball No. 2 '
                              'overall in t

In [320]:
full_article = [convert_story(s1)] + [convert_story(item) for item in related_response.related_stories[:2]]

In [321]:
full_article

[{'text': "D'Angelo Russell's first stint with the Los Angeles Lakers didn't go according to plan as the organization soured on him quickly. With a new front office regime led by Magic Johnson, the Lakers selected Lonzo Ball No. Johnson cited a need for a leader on the team , a direct shot at Russell who was eventually traded to the Brooklyn Nets. Now, Russell's found his way back to Los Angeles and seems ready for a second chance with the team who initially gave up on him. So far, the early returns have been fruitful and even Johnson had to admit that he's seen growth out of Russell, via Bryan Kalbrosky of USA Today “I think D'Angelo has really grown and matured. You have to pick your poison for what you try to do whether it's Russell coming off the PnR or Anthony Davis rolling to the basket and they'll have some more time playing with LeBron. That's for sure.” There's no denying that on the court Russell has become a much better offensive player and already looks like a perfect fit n

In [326]:
 combined_story = {
    'text': '',
    'entities': set(),
    'links': set(),
    'authors': set(),
    'thumbnails': set()
}

for d in full_article:
    combined_story['text'] += d['text']
    combined_story['entities'].update(d['entrities'])
    combined_story['links'].add(d['links'])
    combined_story['authors'].add(d['authors'])
    combined_story['thumbnails'].add(d['thumbnails'])

# combined_story['text'] = get_summary(combined_story['text'], 8)
combined_story
# len(combined_story['entities'])

{'text': "D'Angelo Russell's first stint with the Los Angeles Lakers didn't go according to plan as the organization soured on him quickly. With a new front office regime led by Magic Johnson, the Lakers selected Lonzo Ball No. Johnson cited a need for a leader on the team , a direct shot at Russell who was eventually traded to the Brooklyn Nets. Now, Russell's found his way back to Los Angeles and seems ready for a second chance with the team who initially gave up on him. So far, the early returns have been fruitful and even Johnson had to admit that he's seen growth out of Russell, via Bryan Kalbrosky of USA Today “I think D'Angelo has really grown and matured. You have to pick your poison for what you try to do whether it's Russell coming off the PnR or Anthony Davis rolling to the basket and they'll have some more time playing with LeBron. That's for sure.” There's no denying that on the court Russell has become a much better offensive player and already looks like a perfect fit ne

In [327]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
import wandb

run = wandb.init(project='HeywireAI', entity="heywire")
prediction_table = wandb.Table(columns=["original_story", "prompt", "usage", "completion"])

def generate_story(source_links, entities, summary, words = 500):

    mod = "text-davinci-003"
    prompt = f"Generate a story that involves the following entities and origin example. Please make sure the story you generate is at least {words} words long."

    entity_string = "\n".join([f"- {entity[0]}: {entity[1]}" for entity in entities])
    
    if summary:
        story_prompt = f"{prompt}\nOriginal example: {summary}\n Entities: {entity_string}\n\nBegin story:"
    else:
        story_prompt = f"{prompt}\n{entity_string}\n\nBegin story:"
    
    story_completions = openai.Completion.create(
        model = mod,
        prompt = story_prompt,
        max_tokens=int(4000 - (len(story_prompt) / 3)),
    )
        
    print(story_completions["usage"])
    
    prediction_table.add_data(source_links, story_prompt, story_completions['usage'], story_completions['choices'][0]['text'])
    
    return story_completions['choices'][0]['text']

wandb: Currently logged in as: akurokhtin (heywire). Use `wandb login --relogin` to force relogin


In [330]:
aylien_combined_story = generate_story(
    combined_story['links'], 
    combined_story['entities'], 
    combined_story['text']
)
aylien_combined_story

{
  "completion_tokens": 1244,
  "prompt_tokens": 1167,
  "total_tokens": 2411
}


'\nNorway was abuzz with news of the Los Angeles Lakers’ reunion with D’Angelo Russell, who had originally been traded to the Brooklyn Nets but was now finding his way back to the team that had initially given up on him. With an improved front office, led by Magic Johnson, the team was optimistic to seeing what Russell could bring to the team.\n\nAs teams like Søkeannonser and Forge of Empires - Free Online Game slowly became more popular, CARESOLE became the hottest topic of discussion amongst Lakers fans following their reunion with Russell. While Johnson had high hopes, the Portland Trail Blazers and Joey Mistretta were sceptical.\n\nThe Los Angeles Lakers were in need of a leader, which had initially pointed to Russell. However, Bryan Kalbrosky of USA Today quickly pushed the narrative that, "You have to pick your poison for what you try to do whether it\'s Russell coming off the PnR or Anthony Davis rolling to the basket and they\'ll have some more time playing with LeBron. That\'

In [337]:
# wandb.log({'predictions': prediction_table})
# wandb.finish()
# prediction_table
print(aylien_combined_story)


Norway was abuzz with news of the Los Angeles Lakers’ reunion with D’Angelo Russell, who had originally been traded to the Brooklyn Nets but was now finding his way back to the team that had initially given up on him. With an improved front office, led by Magic Johnson, the team was optimistic to seeing what Russell could bring to the team.

As teams like Søkeannonser and Forge of Empires - Free Online Game slowly became more popular, CARESOLE became the hottest topic of discussion amongst Lakers fans following their reunion with Russell. While Johnson had high hopes, the Portland Trail Blazers and Joey Mistretta were sceptical.

The Los Angeles Lakers were in need of a leader, which had initially pointed to Russell. However, Bryan Kalbrosky of USA Today quickly pushed the narrative that, "You have to pick your poison for what you try to do whether it's Russell coming off the PnR or Anthony Davis rolling to the basket and they'll have some more time playing with LeBron. That's for sur

In [132]:
# import requests
  
# url = 'https://api.aylien.com/news/related_stories?story_id=5333218341'
  
# payload = {}
  
# headers = {
#     'X-Application-ID': 'a7509cd8',
#     'X-Application-Key': '60cdd1a760d58d700bf6b8fb98cfb285'
# }
  
# response = requests.request("GET", url, headers=headers, data = payload)
# res = response.json()
# print(len(res['related_stories']))

10


In [144]:
related_response.related_stories[0]

{'author': {'avatar_url': None, 'id': 3583220, 'name': 'Matt Peralta'},
 'body': "D'Angelo Russell's first stint with the Los Angeles Lakers didn't go "
         'according to plan as the organization soured on him quickly.\n'
         ' Although Russell was clearly a talented player, the organization '
         'had issues with his inconsistent play and his bouts of immaturity. '
         'With a new front office regime led by Magic Johnson, the Lakers '
         'selected Lonzo Ball No. 2 overall in the 2017 NBA Draft. Johnson '
         'cited a need for a leader on the team , a direct shot at Russell who '
         'was eventually traded to the Brooklyn Nets.\n'
         "  Now, Russell's found his way back to Los Angeles and seems ready "
         'for a second chance with the team who initially gave up on him. So '
         'far, the early returns have been fruitful and even Johnson had to '
         "admit that he's seen growth out of Russell, via Bryan Kalbrosky of "
         '

In [152]:
print(related_response.related_stories[0].body)

D'Angelo Russell's first stint with the Los Angeles Lakers didn't go according to plan as the organization soured on him quickly.
 Although Russell was clearly a talented player, the organization had issues with his inconsistent play and his bouts of immaturity. With a new front office regime led by Magic Johnson, the Lakers selected Lonzo Ball No. 2 overall in the 2017 NBA Draft. Johnson cited a need for a leader on the team , a direct shot at Russell who was eventually traded to the Brooklyn Nets.
  Now, Russell's found his way back to Los Angeles and seems ready for a second chance with the team who initially gave up on him. So far, the early returns have been fruitful and even Johnson had to admit that he's seen growth out of Russell, via Bryan Kalbrosky of USA Today
  “I think D'Angelo has really grown and matured. He can score the basketball. He can pass the basketball. He doesn't turn it over a lot. That's what I like about him, too. That pick-and-roll with Anthony Davis is goin

In [148]:
related_response.related_stories[0].id

5333008993

In [153]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def get_summary(article, count):
    LANGUAGE = "english"
    SENTENCES_COUNT = count

    if __name__ == "__main__":
        parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)

        summary = " ".join(str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT))
        return summary

In [160]:
s11_sum = get_summary(related_response.related_stories[0].body, 8)
s11_sum

"D'Angelo Russell's first stint with the Los Angeles Lakers didn't go according to plan as the organization soured on him quickly. Johnson cited a need for a leader on the team , a direct shot at Russell who was eventually traded to the Brooklyn Nets. So far, the early returns have been fruitful and even Johnson had to admit that he's seen growth out of Russell, via Bryan Kalbrosky of USA Today “I think D'Angelo has really grown and matured. You have to pick your poison for what you try to do whether it's Russell coming off the PnR or Anthony Davis rolling to the basket and they'll have some more time playing with LeBron. That's for sure.” There's no denying that on the court Russell has become a much better offensive player and already looks like a perfect fit next to LeBron James and Anthony Davis. Report Ad Off the floor, though, Russell has looked and sounded like a more self-aware person who understands the opportunity in front of him. While it's important for him to figure out hi

In [158]:
import sys
sys.path.append('../api/')
import api

s11_ents = api.text_to_ents(related_response.related_stories[0].body)
s11_ents

[('selge', 'PERSON'),
 ('russell', 'PERSON'),
 ('kneermer', 'PERSON'),
 ('rob pelinka checked', 'PERSON'),
 ('selge bolig', 'PERSON'),
 ('nba draft', 'ORG'),
 ('the brooklyn nets', 'FAC'),
 ('forge of empires', 'ORG'),
 ('2', 'CARDINAL'),
 ('mind darvin', 'PERSON'),
 ('undo\n  anklefit\n  lider', 'ORG'),
 ('forge of empires - free online game', 'PERSON'),
 ('davis', 'PERSON'),
 ('disse', 'GPE'),
 ('caresole\n  ', 'ORG'),
 ('forge of empires - free online game\n  test', 'ORG'),
 ('javascript', 'PERSON'),
 ('søkeannonser', 'PERSON'),
 ("d'angelo", 'PERSON'),
 ('los angeles', 'GPE'),
 ('nesten gitt bort', 'PERSON'),
 ('darvin ham', 'PERSON'),
 ('second', 'ORDINAL'),
 ('norway', 'GPE'),
 ('james', 'PERSON'),
 ('mid-season', 'DATE'),
 ('lakers', 'PERSON'),
 ('lebron', 'PERSON'),
 ('ww iq test\n  sjekk', 'EVENT'),
 ('boligalarmer.no', 'ORG'),
 ('22', 'CARDINAL'),
 ('anthony davis', 'PERSON'),
 ('pnr', 'GPE'),
 ('the new orleans pelicans', 'LOC'),
 ('23', 'CARDINAL'),
 ('youtube', 'ORG'),
 ('